In [197]:
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
from sklearn import metrics

import matplotlib.pyplot as plt
plt.style.use('seaborn-ticks')
%matplotlib inline

In [198]:
df = pd.read_csv("weatherAUS.csv")
print(df.shape)
df.head()

(145460, 23)


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [199]:
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [200]:
target = "RainTomorrow"
numFeatures = [
    "MinTemp",
    "MaxTemp",
    "Rainfall",
    "Evaporation",
    "Sunshine",
    "WindGustSpeed",
    "WindSpeed9am",
    "WindSpeed3pm",
    "Humidity9am",
    "Humidity3pm",
    "Temp9am",
    "Temp3pm",
    "Pressure3pm",
    "Pressure9am",
    "Cloud9am",
    "Cloud3pm",
    "Year",
    "Month",
    "Day",
]
catFeatures = [
    "Location",
    "WindGustDir",
    "WindDir9am",
    "WindDir3pm",
    "RainToday",
    "RainTomorrow",
]


In [201]:
df['Date'] = pd.to_datetime(df['Date'])

In [202]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day


In [203]:
df.drop('Date',inplace= True,axis=1)

In [204]:
df[catFeatures].isnull().sum()

Location            0
WindGustDir     10326
WindDir9am      10566
WindDir3pm       4228
RainToday        3261
RainTomorrow     3267
dtype: int64

In [205]:
df[numFeatures].isnull().sum()

MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustSpeed    10263
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Temp9am           1767
Temp3pm           3609
Pressure3pm      15028
Pressure9am      15065
Cloud9am         55888
Cloud3pm         59358
Year                 0
Month                0
Day                  0
dtype: int64

In [206]:
df["Location"].unique()

array(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'Newcastle', 'NorahHead', 'NorfolkIsland', 'Penrith', 'Richmond',
       'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown',
       'Wollongong', 'Canberra', 'Tuggeranong', 'MountGinini', 'Ballarat',
       'Bendigo', 'Sale', 'MelbourneAirport', 'Melbourne', 'Mildura',
       'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns',
       'GoldCoast', 'Townsville', 'Adelaide', 'MountGambier', 'Nuriootpa',
       'Woomera', 'Albany', 'Witchcliffe', 'PearceRAAF', 'PerthAirport',
       'Perth', 'SalmonGums', 'Walpole', 'Hobart', 'Launceston',
       'AliceSprings', 'Darwin', 'Katherine', 'Uluru'], dtype=object)

In [207]:
pd.get_dummies(df.Location, drop_first=True).head()

,Albany,Albury,AliceSprings,BadgerysCreek,Ballarat,Bendigo,Brisbane,Cairns,Canberra,Cobar,...,Townsville,Tuggeranong,Uluru,WaggaWagga,Walpole,Watsonia,Williamtown,Witchcliffe,Wollongong,Woomera
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [208]:
df = df.join(pd.get_dummies(df.Location, drop_first=True))
df.drop('Location',inplace= True,axis=1)

In [209]:
for col in ['WindGustDir',
 'WindDir9am',
 'WindDir3pm',
 'RainToday',
 'RainTomorrow']:
 df = df[df[col].notna()]

In [210]:
df["WindGustDir"].unique()

array(['W', 'WNW', 'WSW', 'NE', 'NNW', 'N', 'NNE', 'SW', 'ENE', 'SSE',
       'S', 'NW', 'SE', 'ESE', 'E', 'SSW'], dtype=object)

In [211]:
pd.get_dummies(df.WindGustDir, drop_first=True).head()

,ENE,ESE,N,NE,NNE,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [212]:
df = df.join(pd.get_dummies(df.WindGustDir, drop_first=True, prefix="WGD"))
df.drop('WindGustDir', inplace=True, axis=1)

In [213]:
df["WindDir3pm"].unique()

array(['WNW', 'WSW', 'E', 'NW', 'W', 'SSE', 'ESE', 'ENE', 'NNW', 'SSW',
       'SW', 'SE', 'N', 'S', 'NNE', 'NE'], dtype=object)

In [214]:
pd.get_dummies(df.WindDir3pm, drop_first=True).head()

,ENE,ESE,N,NE,NNE,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [215]:
df = df.join(pd.get_dummies(df.WindDir3pm, drop_first=True, prefix="WD3PM"))
df.drop('WindDir3pm',inplace= True,axis=1)

In [216]:
df["WindDir9am"].unique()

array(['W', 'NNW', 'SE', 'ENE', 'SW', 'SSE', 'S', 'NE', 'SSW', 'N', 'WSW',
       'ESE', 'E', 'NW', 'WNW', 'NNE'], dtype=object)

In [217]:
pd.get_dummies(df.WindDir9am, drop_first=True).head()

,ENE,ESE,N,NE,NNE,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [218]:
df = df.join(pd.get_dummies(df.WindDir9am, drop_first=True, prefix="WD9AM"))
df.drop('WindDir9am',inplace= True,axis=1)

In [221]:
df["RainToday"] = df["RainToday"] == 'Yes'

In [222]:
df["RainTomorrow"] = df["RainTomorrow"] == 'Yes'

In [223]:
for x in list(df.columns.values):
    df[x] = df[x].fillna(df[x].mean())

In [ ]:
df['winter'] = df.apply(
    lambda row: row.Month == 12 or row.Month == 1 or row.Month == 2, axis=1)
df['spring'] = df.apply(
    lambda row: row.Month == 3 or row.Month == 4 or row.Month == 5, axis=1)
df['summer'] = df.apply(
    lambda row: row.Month == 6 or row.Month == 7 or row.Month == 8, axis=1)
df['autumn'] = df.apply(
    lambda row: row.Month == 9 or row.Month == 10 or row.Month == 11, axis=1)

In [ ]:
df['winterRain'] = df.apply(
    lambda row: row.RainToday == True and row.winter == True, axis=1)
df['springRain'] = df.apply(
    lambda row: row.RainToday == True and row.spring == True, axis=1)
df['summerRain'] = df.apply(
    lambda row: row.RainToday == True and row.summer == True, axis=1)
df['autumnRain'] = df.apply(
    lambda row: row.RainToday == True and row.autumn == True, axis=1)

In [224]:
df

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WD9AM_NNW,WD9AM_NW,WD9AM_S,WD9AM_SE,WD9AM_SSE,WD9AM_SSW,WD9AM_SW,WD9AM_W,WD9AM_WNW,WD9AM_WSW
0,13.4,22.9,0.6,5.609501,7.718384,44.0,20.0,24.0,71.0,22.0,...,0,0,0,0,0,0,0,1,0,0
1,7.4,25.1,0.0,5.609501,7.718384,44.0,4.0,22.0,44.0,25.0,...,1,0,0,0,0,0,0,0,0,0
2,12.9,25.7,0.0,5.609501,7.718384,46.0,19.0,26.0,38.0,30.0,...,0,0,0,0,0,0,0,1,0,0
3,9.2,28.0,0.0,5.609501,7.718384,24.0,11.0,9.0,45.0,16.0,...,0,0,0,1,0,0,0,0,0,0
4,17.5,32.3,1.0,5.609501,7.718384,41.0,7.0,20.0,82.0,33.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145454,3.5,21.8,0.0,5.609501,7.718384,31.0,15.0,13.0,59.0,27.0,...,0,0,0,0,0,0,0,0,0,0
145455,2.8,23.4,0.0,5.609501,7.718384,31.0,13.0,11.0,51.0,24.0,...,0,0,0,1,0,0,0,0,0,0
145456,3.6,25.3,0.0,5.609501,7.718384,22.0,13.0,9.0,56.0,21.0,...,0,0,0,1,0,0,0,0,0,0
145457,5.4,26.9,0.0,5.609501,7.718384,37.0,9.0,9.0,53.0,24.0,...,0,0,0,1,0,0,0,0,0,0


In [225]:
features = df.columns.tolist()
features.remove(target)
print(features)

['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 'RainToday', 'Year', 'Month', 'Day', 'Albany', 'Albury', 'AliceSprings', 'BadgerysCreek', 'Ballarat', 'Bendigo', 'Brisbane', 'Cairns', 'Canberra', 'Cobar', 'CoffsHarbour', 'Dartmoor', 'Darwin', 'GoldCoast', 'Hobart', 'Katherine', 'Launceston', 'Melbourne', 'MelbourneAirport', 'Mildura', 'Moree', 'MountGambier', 'MountGinini', 'Newcastle', 'Nhil', 'NorahHead', 'NorfolkIsland', 'Nuriootpa', 'PearceRAAF', 'Penrith', 'Perth', 'PerthAirport', 'Portland', 'Richmond', 'Sale', 'SalmonGums', 'Sydney', 'SydneyAirport', 'Townsville', 'Tuggeranong', 'Uluru', 'WaggaWagga', 'Walpole', 'Watsonia', 'Williamtown', 'Witchcliffe', 'Wollongong', 'Woomera', 'WGD_ENE', 'WGD_ESE', 'WGD_N', 'WGD_NE', 'WGD_NNE', 'WGD_NNW', 'WGD_NW', 'WGD_S', 'WGD_SE', 'WGD_SSE', 'WGD_SSW', 'WGD_SW', 'WGD_W', 'WGD_

In [226]:
df[target] = df[target].astype(int)

In [227]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

# Przygotujmy prosty wrapper
def CVTest(nFolds = 5, randomState=2020, features=[]):
    kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)

    # Aby oszczędzać pamięć informacja o foldach to wyłącznie numery wierszy
    testResults = []
    for train, test in kf.split(df.index.values):
        mod = sm.GLM.from_formula(formula=target + " ~ "+ "+".join(features),
                                  data=df.iloc[train], family=sm.families.Binomial())
        res = mod.fit()
        predsTrain = res.predict(df.iloc[train])
        preds = res.predict(df.iloc[test])
        testResults.append(roc_auc_score(df[target].iloc[test], preds))
        
    return np.mean(testResults)

In [228]:
CVTest(features=final_features)

PatsyError: Error evaluating factor: NameError: name 'spring' is not defined
    RainTomorrow ~ MinTemp+MaxTemp+Rainfall+Evaporation+Sunshine+WindGustSpeed+WindSpeed9am+WindSpeed3pm+Humidity9am+Humidity3pm+Pressure9am+Pressure3pm+Cloud9am+Cloud3pm+Temp9am+Temp3pm+RainToday+Year+Month+Day+winter+spring+summer+autumn+winterRain+springRain+summerRain+autumnRain
                                                                                                                                                                                                                           ^^^^^^

In [ ]:
from scipy import stats
from sklearn import feature_selection

In [ ]:
minfos= []
for var in features:
    print("Values for pair {0}".format(var))
    print("Spearman", stats.spearmanr(df[target], df[var]))
    print("Mutual info", feature_selection.mutual_info_classif(df[var].values.reshape(-1,1),df[target].values))
    minfos.append(feature_selection.mutual_info_classif(df[var].values.reshape(-1,1),df[target].values)[0])

imp = list(zip(minfos, features))
imp.sort(reverse=True)
imp

In [ ]:
df.winterRain.value_counts()

In [ ]:
final_features = ['MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustSpeed',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Cloud9am',
 'Cloud3pm',
 'Temp9am',
 'Temp3pm',
 'RainToday',
 'Year',
 'Month',
 'Day',
 'winter',
 'spring',
 'summer',
 'autumn',
 'winterRain',
 'springRain',
 'summerRain',
 'autumnRain']